put train-test infront of files accordingly

input layer:
-MFCC features

hidden layer:
-neurons only

output layer:
-how many environments labels



In [2]:
import os
import tkinter as tk
from tkinter import filedialog


In [3]:
# essential for using opening files UI
root = tk.Tk()
root.attributes('-topmost', True)  # makes it appear on top of other windows
root.update()
root.withdraw()

''

In [4]:
try:
    from pydub import AudioSegment as am
except ImportError as e:
    import pip
    pip.main(["install","pydub"])
    from pydub import AudioSegment as am

In [9]:
# split car.ogg file into two 15 minute chunks and change format to *.wav file
carFile = filedialog.askopenfilename(parent=root,initialdir=os.path.dirname(__name__))
rawData = filedialog.askdirectory(parent=root,initialdir=os.path.dirname(__name__))
minute = 60*1000

sound = am.from_file(carFile)

# splitting...
sound[-30*minute:-15*minute].export(os.path.join(rawData,"car_1.ogg"),format="ogg")
sound[-15*minute:].export(os.path.join(rawData,"car_2.ogg"),format="ogg")

<_io.BufferedRandom name='S:/Projects/Python/GraduationProject/RawData\\car_2.ogg'>

In [ ]:
# sounds files can be sliced using milliseconds
_15min = 15*60*1000
for i in wav_files:
    # taking the last 15 minutes from each file
    sound = am.from_file(os.path.join(i))
    sound = sound.set_frame_rate(16000)
    sound[-_15min:].export(os.path.join(rawData_folder,i),format="wav")


In [ ]:
for i in wav_files:
    sound = am.from_file(i)
    print(i,":",len(sound))

S:\Projects\Python\GraduationProject\firstFifty\airPort_1.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\airPort_2.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\airport_3.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\car_1.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\car_2.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\car_3.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\publicRoad_1.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\trainStation_1.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\trainStation_2.wav : 450000
S:\Projects\Python\GraduationProject\firstFifty\trainStation_3.wav : 450000
S:\Projects\Python\GraduationProject\secondFifty\airPort_1.wav : 450000
S:\Projects\Python\GraduationProject\secondFifty\airPort_2.wav : 450000
S:\Projects\Python\GraduationProject\secondFifty\airport_3.wav : 450000
S:\Projects\Python\GraduationProject\secondFifty\car_1.wav : 450000


In [ ]:


try:
    os.mkdir("firstFifty")
    os.mkdir("secondFifty")
except OSError:
    print("Already exist")
for file_path in wav_files:
    sound = am.from_file(file_path)
    file_path = file_path.split("/")[-1] # /home/extacy/Desktop/GraduationProject/wavFiles/car_2.wav => car_2.wav
    firstFifty = os.path.join("firstFifty",file_path)
    secondFifty = os.path.join("firstFifty",file_path)
    
    sound[:len(sound)//2].export(firstFifty,format="wav")
    sound[len(sound)//2:].export(secondFifty,format="wav")

Already exist
